In [1]:
import fit_tools
from fitparse import FitFile
import pandas as pd
import numpy as np
from bokeh.models import Toggle, BoxAnnotation, CustomJS


In [3]:
fit = FitFile('2017-08-08-07-28-06.fit')
fit.parse()

df = fit_tools.FitToDataFrame(fit, ['power'])


In [4]:
df = fit_tools.FitPowerCurve(fit)

In [5]:
df.head()

,0
timestamp,
2017-08-08 11:28:06,815.000000
2017-08-08 11:28:07,774.500000
2017-08-08 11:28:08,659.666667
2017-08-08 11:28:09,594.000000
2017-08-08 11:28:10,572.800000


In [6]:
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import HoverTool, BoxZoomTool, ResetTool

# output to static HTML file
output_notebook()

# create a new plot with a title and axis labels
p = figure(title="Power Curve", width=800, height=400, 
           x_axis_type="log", x_axis_label='time',
           y_axis_label='power', tools=[BoxZoomTool(), ResetTool()])

power_data = df[0]
x = [i for i in range(0, len(power_data))]
# add a line renderer with legend and line thickness
p.line(x, power_data, legend="Power", line_width=2)

# show the results
show(p)

Loading BokehJS ...

In [8]:
intervals = fit_tools.FitIntervals(fit)
intervals.Find(360, 4)
intervals.Find(120, 4)

hover = HoverTool(names=['avg_power'],
    tooltips=[
        ("Power", "@top"),
        ("Heart Rate", "@hr")
    ]
)

# create a new plot with a title and axis labels
p = figure(title="power", width=800, height=240, 
           x_axis_type="datetime", x_axis_label='time', 
           y_axis_label='power', tools=[hover, BoxZoomTool(), ResetTool()])

fdf = intervals.df
power_data = fdf['power']
hr_data = fdf['heart_rate']

# do some smoothing just for prettier rendering
window_size = 60
window = np.ones(window_size)/float(window_size)
power_avg = np.convolve(power_data, window, 'same')

# add a line renderer with legend and line thickness
p.line(fdf.index, power_avg, legend="Power", line_width=2)
p.line(fdf.index, fdf['heart_rate'], color='red', legend="HR", line_width=1)

# convert interval data into ColumnDataSource
idx = 0
left = [i.start for i in intervals]
top = [int(power_data[i.start:i.end].mean()) for i in intervals]
right = [i.end for i in intervals.intervals]
bottom = [0 for i in intervals.intervals]
hr = [int(hr_data[i.start:i.end].mean()) for i in intervals]

source = ColumnDataSource({"top":top, "right":right, "bottom":bottom,"left":left, "hr":hr})
p.quad(top="top", bottom="bottom", right="right", left="left", 
       source=source, color="green", fill_alpha=0.2, name="avg_power")

# show the results
show(p)